In [726]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import calendar
import matplotlib.pyplot as plt
from collections import Counter

In [727]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(2)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
838,tt0881320,30000000,108609310,Sanctum,Ioan Gruffudd|Richard Roxburgh|Alice Parkinson|Daniel Wyllie|Allison Cratchley,Alister Grierson,The only way out is down.,"The 3-D action-thriller Sanctum, from executive producer James Cameron, follows a team of underwater cave divers on a treacherous expedition to the largest, most beautiful and least accessible cave system on Earth. When a tropical storm forces them deep into the caverns, they must fight raging water, deadly terrain and creeping panic as they search for an unknown escape route to the sea. Master diver Frank McGuire (Richard Roxburgh) has explored the South Pacific's Esa-ala Caves for months. But when his exit is cut off in a flash flood, Frank's team--including 17-year-old son Josh (Rhys Wakefield) and financier Carl Hurley (Ioan Gruffudd)--are forced to radically alter plans. With dwindling supplies, the crew must navigate an underwater labyrinth to make it out. Soon, they are confronted with the unavoidable question: Can they survive, or will they be trapped forever?",108,Action|Thriller,Universal Pictures|Wayfare Entertainment|Sanctum Australia|Relativity Media|Great Wight Productions/ Osford Films,2/3/2011,5.7,2011
1502,tt0449059,8000000,100523181,Little Miss Sunshine,Greg Kinnear|Toni Collette|Steve Carell|Paul Dano|Abigail Breslin,Jonathan Dayton|Valerie Faris,A family on the verge of a breakdown,"A family loaded with quirky, colorful characters piles into an old van and road trips to California for little Olive to compete in a beauty pageant.",102,Comedy|Drama,Fox Searchlight Pictures|Bona Fide Productions|Big Beach Films,7/26/2006,7.3,2006


In [728]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [729]:
answers = {} # создадим словарь для ответов

#regex patterns
monthExtractionPattern = re.compile('^([0-9]{1,2})/[0-9]{1,2}/[0-9]{4}$')
alphaNumericPattern = re.compile('[\W_]+')

#value generator functions
def extractMonth(date):
    return monthExtractionPattern.search(date).group(1)

def extractWordsCount(title):
    return len(list(filter(lambda w: len(alphaNumericPattern.sub('', w)) > 1, title.split(' '))))

def explode(df, columns):
    temp = df
    for c in columns:
        temp = temp.explode(c)
    return temp

#new data columns
data['title_with_id'] = data.apply(lambda d: f'{d.original_title} ({d.imdb_id})', axis = 1)
data['release_month'] = data.release_date.apply(extractMonth)
data['profit'] = data['revenue'] - data['budget']
data['is_profitable'] = data['profit'] > 0
data['exploded_genres'] = data.genres.apply(lambda g: g.split('|'))
data['exploded_director'] = data.director.apply(lambda d: d.split('|'))
data['actor'] = data.cast.apply(lambda d: d.split('|'))
data['company'] = data.production_companies.apply(lambda d: d.split('|'))

#data unnesting: warn! exploded data frame would contain duplicated values
exploded_data = explode(data, ['exploded_genres', 'exploded_director', 'actor','company'])
exploded_data['title_length'] = exploded_data.original_title.str.len()
exploded_data['overview_word_count'] = exploded_data.overview.apply(extractWordsCount)

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [730]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
##+

In [731]:
##most_expensive_movie = data.sort_values(by = 'budget', ascending=False).head(1)
most_expensive_movie = data.loc[data['budget'].argmax()]
most_expensive_movie.title_with_id

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

# 2. Какой из фильмов самый длительный (в минутах)?

In [732]:
answers['2'] = 'Gods and Generals (tt0279111)'
##+

In [733]:
longest_movie = data.loc[data.runtime.argmax()]
longest_movie.title_with_id

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [734]:
answers['3'] = 'Winnie the Pooh (tt1449283)'
##+

In [735]:
shortest_movie = data.loc[data.runtime.argmin()]
shortest_movie.title_with_id

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [736]:
answers['4'] = '110'
##+

In [737]:
round(data.mean().runtime)

110.0

# 5. Каково медианное значение длительности фильмов? 

In [738]:
answers['5'] = '107'
##+

In [739]:
round(data.median().runtime)

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [740]:
answers['6'] = 'Avatar (tt0499549)'
##+

In [741]:
most_profitable_movie = data.loc[data.profit.argmax()]
most_profitable_movie.title_with_id

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [742]:
answers['7'] = 'The Lone Ranger (tt1210819)'
##+

In [743]:
least_profitable_movie = data.loc[data.profit.argmin()]
least_profitable_movie.title_with_id

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [744]:
answers['8'] = '1478'
##+

In [745]:
data.query('profit > 0').imdb_id.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [746]:
answers['9'] = 'The Dark Knight (tt0468569)'
##+

In [747]:
movies_2008 = data.query('release_year == "2008"').reset_index()
most_profitable_2008_movie = movies_2008.loc[movies_2008.profit.argmax()]
most_profitable_2008_movie.title_with_id

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [748]:
answers['10'] = 'The Lone Ranger (tt1210819)'
##+

In [749]:
mid_2010s_movies = data.query('release_year in ("2012", "2013", "2014")').reset_index()
worst_of_2012_2014_movie = mid_2010s_movies.loc[mid_2010s_movies.profit.argmin()]
worst_of_2012_2014_movie.title_with_id

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [750]:
answers['11'] = 'Drama'
##+

In [751]:
genres_stats = exploded_by_genre_data.groupby('exploded_genres').imdb_id.nunique().reset_index()
top_genre = genres_stats.loc[genres_stats.imdb_id.argmax()]
top_genre.exploded_genres

'Drama'

ВАРИАНТ 2

In [752]:
most_popular_genre = exploded_by_genre_data.groupby('exploded_genres').imdb_id.nunique().sort_values(ascending=False).head(1)
genre, *_ = most_popular_genre.keys()
genre

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [753]:
answers['12'] = 'Drama'
##+

In [754]:
another_genres_stat = exploded_data.query('is_profitable').groupby('exploded_genres').imdb_id.nunique().reset_index()
most_profitable_genres = another_genres_stat.loc[another_genres_stat.imdb_id.argmax()]
most_profitable_genres.exploded_genres

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [755]:
answers['13'] = 'Peter Jackson'
##+

In [756]:
director_revenue_stat = data.groupby('director').revenue.sum().reset_index()
most_profitable_director = director_revenue_stat.loc[director_revenue_stat.revenue.argmax()]
most_profitable_director.director

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [757]:
answers['14'] = 'Robert Rodriguez'
##+

In [758]:
action_movies = exploded_data[exploded_data.genres.str.contains('Action')]
action_hero_directors = action_movies.groupby('exploded_director').imdb_id.nunique().reset_index()
action_hero_director = action_hero_directors.loc[action_hero_directors.imdb_id.argmax()]
action_hero_director.exploded_director

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [759]:
answers['15'] = 'Chris Hemsworth'
##+

In [760]:
movies_2012 = exploded_data.query('release_year == "2012"')[['imdb_id','actor','revenue']].drop_duplicates()
best_actors_stats = movies_2012.groupby('actor').revenue.sum().reset_index()
best_actor = best_actors_stats.loc[best_actors_stats.revenue.argmax()]
best_actor.actor

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [761]:
answers['16'] = 'Matt Damon'
##+

In [762]:
mean_budget = data.budget.mean()
actor_budget_stats = exploded_data.query(f'budget > {mean_budget}').groupby('actor').imdb_id.nunique().reset_index()
best_actor = actor_budget_stats.loc[actor_budget_stats.imdb_id.argmax()]
best_actor.actor


'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [763]:
answers['17'] = 'Action'
##+

In [764]:
nicolas_cage_movies = exploded_data.query('actor == "Nicolas Cage"')
nic_cage_genres = nicolas_cage_movies.groupby('exploded_genres').imdb_id.nunique().reset_index()
nic_top_genre = nic_cage_genres.loc[nic_cage_genres.imdb_id.argmax()]
nic_top_genre.exploded_genres

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [765]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'
##+

In [766]:
paramount_movies = exploded_data.query('company == "Paramount Pictures"').reset_index()
worst_paramount_movie = paramount_movies.loc[paramount_movies.profit.argmin()]
worst_paramount_movie.title_with_id

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [767]:
answers['19'] = '2015'
##+

In [768]:
year_stats = data.groupby('release_year').revenue.sum().reset_index()
best_year = year_stats.loc[year_stats.revenue.argmax()]
best_year.release_year

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [769]:
answers['20'] = '2014'
##+

In [770]:
warner_mask = exploded_data.company.str.contains('Warner Bros')
warner_movies = exploded_data[warner_mask][['release_year','imdb_id', 'profit']].drop_duplicates()
warner_stats = warner_movies.groupby('release_year').profit.sum().reset_index()
best_warner_year = warner_stats.loc[warner_stats.profit.argmax()]
best_warner_year.release_year

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [771]:
answers['21'] = 'September'
##+

In [772]:
month_stats = data.groupby('release_month').imdb_id.nunique().reset_index()
top_month_num = month_stats.loc[month_stats.imdb_id.argmax()].release_month
calendar.month_name[int(top_month_num)]

'September'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [773]:
answers['22'] = '450'
##+

In [774]:
data.query('release_month in ("6","7","8")').imdb_id.nunique()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [775]:
answers['23'] = 'Peter Jackson'
##+

In [776]:
winter_stats = exploded_data.query('release_month in ("1","2","12","01","02")').groupby('exploded_director').imdb_id.nunique().reset_index()
top_winter_director = winter_stats.loc[winter_stats.imdb_id.argmax()]
top_winter_director.exploded_director

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [777]:
answers['24'] = 'Four By Two Productions'
#+

In [778]:
company_titles = exploded_data[['imdb_id','company','title_length']].drop_duplicates()
company_titles_stat = company_titles.groupby('company').title_length.mean().reset_index()
company_titles_stat.loc[company_titles_stat.title_length.argmax()].company

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [779]:
answers['25'] = 'Midnight Picture Show'
##+

In [780]:
mean_desc_words = exploded_data[['imdb_id', 'company', 'overview_word_count']].drop_duplicates().groupby('company').overview_word_count.mean().reset_index()
wordy_company = mean_desc_words.loc[mean_desc_words.overview_word_count.argmax()].company
wordy_company

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [781]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
##+

In [782]:
vote_99th = data.vote_average.quantile(0.99)
top_1pct_movies = data.query(f'vote_average >= {vote_99th}')
top_1pct_count = top_1pct_movies.imdb_id.nunique()
top_1pct_movies.sort_values(by = 'vote_average', ascending=False).original_title.head(top_1pct_count)

599     The Dark Knight                                  
9       Inside Out                                       
34      Room                                             
118     Interstellar                                     
125     The Imitation Game                               
370     Inception                                        
1191    12 Years a Slave                                 
1183    The Wolf of Wall Street                          
1081    The Lord of the Rings: The Return of the King    
872     The Pianist                                      
1800    Memento                                          
119     Guardians of the Galaxy                          
138     The Grand Budapest Hotel                         
128     Gone Girl                                        
283     Mr. Nobody                                       
1688    There Will Be Blood                              
1563    Eternal Sunshine of the Spotless Mind            
1444    The Pr

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [783]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
##+

In [784]:
def actorsPair(actor1, actor2):
    stable = sorted([actor1, actor2])
    return f'{stable[0]} & {stable[1]}'
def actorsPairFromRow(row):
    return actorsPair(row.actor_x, row.actor_y)

In [789]:
def composeActorMoviePairs(row):
    pairs = []
    for i, a1 in enumerate(row.actor, start=0):
        for a2 in row.actor[i+1:]:
            if a1 != a2:
                pairs.append(actorsPair(a1, a2))
    return pairs
data['actor_pair'] = data.apply(composeActorMoviePairs, axis=1)
exploded_pairs = data[['imdb_id','original_title','actor_pair']].explode('actor_pair')
another_pairs_score = exploded_pairs.groupby('actor_pair').imdb_id.nunique().reset_index()
max_pair_score = another_pairs_score.imdb_id.max()
another_pairs_score.query(f'imdb_id == {max_pair_score}')

,actor_pair,imdb_id
7007,Daniel Radcliffe & Emma Watson,8
7023,Daniel Radcliffe & Rupert Grint,8
9143,Emma Watson & Rupert Grint,8


ВАРИАНТ 2

In [786]:
movie_to_actor = exploded_data[['imdb_id', 'actor']].drop_duplicates()
movie_pairs = movie_to_actor.merge(movie_to_actor, on = 'imdb_id', how = 'inner').query('actor_x != actor_y')
movie_pairs['pair'] = movie_pairs.apply(actorsPairFromRow, axis=1)
pair_scores = movie_pairs.groupby('pair').imdb_id.nunique().reset_index()
most_consistent_pair_score = pair_scores.loc[pair_scores.imdb_id.argmax()].imdb_id
pair_scores.query(f'imdb_id == {most_consistent_pair_score}')

,pair,imdb_id
7007,Daniel Radcliffe & Emma Watson,8
7023,Daniel Radcliffe & Rupert Grint,8
9143,Emma Watson & Rupert Grint,8


# Submission

In [787]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [788]:
# и убедиться что ни чего не пропустил)
len(answers)

27